<a href="https://colab.research.google.com/github/MoonJaeHoon/Dacon_Behavioral_Data_Analysis/blob/master/%EB%B2%88%EC%99%B8_Camera%EB%B3%80%EC%88%98%EC%97%90_%EB%8C%80%ED%95%B4%EC%84%9CCounterVectorizer_digitizer%ED%99%9C%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
print( '변경 전 기본 경로 :' )
print(  os.getcwd() )
os.chdir('./drive/My Drive/데이콘_게임데이터분석')
print( '변경 후 파일 경로 :' )
print(  os.getcwd()  )

import numpy as np
import pandas as pd

from tqdm import tqdm_notebook              # 진행바
from sklearn.metrics import roc_auc_score   # AUC 스코어 계산
from sklearn.model_selection import KFold   # K-fold CV    
#from bayes_opt import BayesianOptimization  # 베이지안 최적화 라이브러리  
from functools import partial               # 함수 변수 고정
import lightgbm as lgb                      # LightGBM 라이브러리
import warnings                             
warnings.filterwarnings("ignore")           # 경고 문구 미표시
%time
train = pd.read_csv('./train.csv')

#%time
#test = pd.read_pickle('./test.pkl')

변경 전 기본 경로 :
/content
변경 후 파일 경로 :
/content/drive/My Drive/데이콘_게임데이터분석
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs


## Train 으로부터 mytrain 데이터 생성

In [0]:
Camera = train.loc[train['event']=='Camera',:]
game_id = train.game_id.unique().tolist()
winner =  train.loc[train['player']==0,:].groupby(    ['game_id'] )['winner'].max()
del train

In [0]:
Camera['event_contents'].value_counts()

at (143.5, 22.7578125)          89727
at (24.5, 145.7578125)          88246
at (0.0, 0.0)                   82567
at (33.5, 23.7578125)           73001
at (26.5, 126.76171875)         71466
                                ...  
at (131.4609375, 32.1171875)        1
at (30.08203125, 103.59375)         1
at (32.34375, 99.8046875)           1
at (34.1640625, 18.75)              1
at (150.8984375, 78.3671875)        1
Name: event_contents, Length: 23902568, dtype: int64

In [0]:
import re
x = "at (143.5, 22.7578125)"
interval = 
result = re.sub("['()]","",x)
result = re.sub('at','', result)
result = re.sub(' ','', result)
ran = int(180/interval)
#location0 = np.digitize(    round(   float(   result.split(',')[0]    )  ), bins=[i*interval for i in range(36)]   )
location0 = str(np.digitize(    round(    float(   result.split(',')[0]    )  ), bins=[i*interval for i in range(ran)]   ))
#location1 = np.digitize(    round(   float(   result.split(',')[1]    )    ), bins=[i*interval for i in range(36)]   )
location1 = str(np.digitize(    round(    float(   result.split(',')[1]    )  ), bins=[i*interval for i in range(ran)]   ))
result = location0 + '_' + location1
result

'29_5'

In [0]:
import re
#def SUB(x):
#    result = re.sub("['()]","",x)
#    result = re.sub('at','', result)
#    result = re.sub("[,]","_",result)
#    result = re.sub(' ','', result)
#    result = result + ','
#    return result

def SUB(x):
    interval = 10
    result = re.sub("['()]","",x)
    result = re.sub('at','', result)
    result = re.sub(' ','', result)
    ran = int(180/interval)
    #location0 = str(    int(round(   float(   result.split(',')[0]    )  ) )    )
    location0 = str(np.digitize(    round(    float(   result.split(',')[0]    )  ), bins=[i*interval for i in range(ran)]   ))
    #location1 = str(    int(round(   float(   result.split(',')[1]    )  ) )   )
    location1 = str(np.digitize(    round(    float(   result.split(',')[1]    )  ), bins=[i*interval for i in range(ran)]   ))
    result = location0 + '_' + location1
    del location0, location1
    result = result + ','
    return result

Camera['event_contents'] = Camera.apply(lambda x: SUB(x['event_contents']), axis=1)
Camera['event_contents'].value_counts()

15_3,    1368738
3_13,    1071850
4_3,      959420
3_12,     890975
4_13,     866083
          ...   
4_1,          24
2_4,          15
2_17,         11
2_3,          10
2_2,           1
Name: event_contents, Length: 284, dtype: int64

In [0]:
a= Camera['event_contents'].value_counts()

In [0]:
len(a.loc[a>10000])

357

In [0]:
del a

In [0]:

columns = ['text0','text1','winner']
mytrain = np.zeros(len(game_id)*len(columns))
mytrain = mytrain.reshape(len(game_id),len(columns))
mytrain = pd.DataFrame(mytrain, columns=columns)
mytrain.index = game_id
mytrain.winner = winner
mytrain.loc[:,'text0'] = Camera.loc[Camera['player']==0,:].loc[Camera['event']=='Camera',:].groupby(['game_id']).event_contents.sum()
mytrain.loc[:,'text1'] = Camera.loc[Camera['player']==1,:].loc[Camera['event']=='Camera',:].groupby(['game_id']).event_contents.sum()
del Camera
mytrain = mytrain.fillna('')


## Test 로부터 mytest 데이터 생성

In [0]:
test = pd.read_csv('test.csv')
game_id = test.game_id.unique().tolist()
Camera = test.loc[test['event']=='Camera',:]
del test

Camera['event_contents'] = Camera.apply(lambda x: SUB(x['event_contents']), axis=1)
Camera['event_contents'].value_counts()

15_3,    579061
3_13,    450045
4_3,     404225
3_12,    374832
4_13,    370596
          ...  
5_1,         10
2_2,          9
2_3,          6
2_4,          3
2_17,         2
Name: event_contents, Length: 284, dtype: int64

In [0]:
columns = ['text0','text1']
mytest = np.zeros(len(game_id)*len(columns))
mytest = mytest.reshape(len(game_id),len(columns))
mytest = pd.DataFrame(mytest, columns=columns)
mytest.index = game_id

mytest.loc[:,'text0'] = Camera.loc[Camera['player']==0,:].loc[Camera['event']=='Camera',:].groupby(['game_id']).event_contents.sum()
mytest.loc[:,'text1'] = Camera.loc[Camera['player']==1,:].loc[Camera['event']=='Camera',:].groupby(['game_id']).event_contents.sum()
del Camera
mytest = mytest.fillna('')


## PipeLine 없이 모델링

### 1. CountVevtorizer

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
def tokenizer(text):
    return text.split(',')

########### 사용할 모수들 ###################
min_df = 3000
ngram_range = (1,1)
seed = 42

######################## 데이터로부터 단어가방 생성 ####################
vectorizer0 = CountVectorizer(min_df=min_df, ngram_range=ngram_range)
revised_reviews0 = [' '.join(review.split(',')) for review in mytrain.text0]
vector_reviews0 = vectorizer0.fit_transform(revised_reviews0)
print(vector_reviews0.shape)

vectorizer1 = CountVectorizer(min_df=min_df, ngram_range=ngram_range)
revised_reviews1 = [' '.join(review.split(',')) for review in mytrain.text1]
vector_reviews1 = vectorizer1.fit_transform(revised_reviews1)
print(vector_reviews1.shape)

player0 = pd.DataFrame(   vector_reviews0.toarray()*1, columns = vectorizer0.get_feature_names()  )
player0 = player0.add_prefix('player0_')
player1 = pd.DataFrame(   vector_reviews1.toarray()*1, columns = vectorizer1.get_feature_names()  )
player1 = player1.add_prefix('player1_')
X = pd.concat([player0,player1],axis=1)
y = mytrain.winner
print(X.shape)
del vectorizer0, vectorizer1, revised_reviews0, revised_reviews1, vector_reviews0, vector_reviews1
del player0, player1


(38872, 164)
(38872, 166)
(38872, 330)


In [0]:
#del X_train, X_valid, y_train, y_valid
########### 사용할 모수들 ###################
min_df = 1
ngram_range = (1,1)
seed = 42

######################## 데이터로부터 단어가방 생성 ####################
vectorizer0 = CountVectorizer(min_df=min_df, ngram_range=ngram_range)
revised_reviews0 = [' '.join(review.split(',')) for review in mytest.text0]
vector_reviews0 = vectorizer0.fit_transform(revised_reviews0)
print(vector_reviews0.shape)

vectorizer1 = CountVectorizer(min_df=min_df, ngram_range=ngram_range)
revised_reviews1 = [' '.join(review.split(',')) for review in mytest.text1]
vector_reviews1 = vectorizer1.fit_transform(revised_reviews1)
print(vector_reviews1.shape)

player0 = pd.DataFrame(   vector_reviews0.toarray()*1, columns = vectorizer0.get_feature_names()  )
player0 = player0.add_prefix('player0_')
player1 = pd.DataFrame(   vector_reviews1.toarray()*1, columns = vectorizer1.get_feature_names()  )
player1 = player1.add_prefix('player1_')

#del vectorizer0, vectorizer1, revised_reviews0, revised_reviews1, vector_reviews0, vector_reviews1

X_test = pd.concat([player0,player1],axis=1)
print(X_test.shape)

#del player0, player1

(16787, 281)
(16787, 283)
(16787, 564)


In [0]:
print(len(X.columns))
print(len(X_test.columns))
print(  len(    set(X.columns) & set(X_test.columns)   ) )

use_col = list(    set(X.columns) & set(X_test.columns)   )

X = X.loc[:,use_col]
X_test = X_test.loc[:,use_col]

330
564
330


#X = pd.read_pickle('train_Camera_10단위.pkl')
#X_test = pd.read_pickle('test_Camera_10단위.pkl')
print(X.shape)
print(X_test.shape)

In [0]:

###################### 모델링 시작 #######################
X_train, X_valid, y_train, y_valid = train_test_split(X, y,   stratify=y, random_state=seed)
from sklearn.naive_bayes import BernoulliNB
#model_bern = BernoulliNB().fit(X_train, y_train)
from sklearn.naive_bayes import MultinomialNB
#MNB = MultinomialNB(alpha=1).fit(X_train,y_train)
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=500, random_state=seed).fit(X_train, y_train)
print('랜포 학습완료')
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(n_estimators=500,random_state=seed).fit(X_train,y_train)
print('LGBM 학습완료')

랜포 학습완료
LGBM 학습완료


In [0]:
## 랜포     #raw_  min_df=1000 & 300 : 0.5661 (941개)
## int:     min_df=2500 & 500 : 0.5373 (748),   3000 & 1000 : 0.5270 (387)
## 3단위:   min_df=5000 & 500 : 0.5421 (328)
## 5단위:   min_df=2000 & 200 : 0.5712 (899),    2500&250 : 0.5712 (718),   4000&250 : 0.5648 (428), 5000&250 : 0.5504(285) ###############################################
## 6단위:   min_df= 2000 & 200  : 0.5721 (753),  2500&250 : 0.5714 (634), 5000&500 : 0.5636 (255)   ############################################
## 7.5단위:   min_df= 1 & 1 : 0.5658 (905)
## 10단위:  min_df= 1 : 0.5703 (564) ,  5 : 0.5685 (560),   10 : 0.5673(555), 2000 : 0.5722 (397), 3000 : 0.5711 (330),  (286), 4000 : 0.5733 (243)
## 12단위:  min_df= 1 & 1 : 0.5660 (400),          #####     12 & 1 : 0.5725(391) ,#####             13 :0.5678 (389), 14 : 0.5689
## 15단위:  min_df= 1 & 1 : 0.5645 (256), 5 : 0.5614
y_pred = rf.predict_proba(X_valid)[:,1]
print(rf)
print("테스트 정확도: {:.4f}".format(roc_auc_score(y_valid, y_pred)))

## LGBM     #raw_  min_df=1000 & 300 : 0.5518 (941개)
## int:   0.5323,   0.5267
## 3단위: 0.5278
## 5단위: 0.5523 (899), 0.5463, 0.5468, 0.5350
## 6단위: 0.5380,0.5410,0.0.5414
## 7.5단위: 0.5299
## 10단위: 0.5511, 2000 : 0.5527, 3000 : 0.5590, 4000 : 0.5515
## 12단위: 0.5452, 0.5452, 0.5452, 0.5452, 
## 15단위: 
y_pred = lgbm.predict_proba(X_valid)[:,1]
print(lgbm)
print("테스트 정확도: {:.4f}".format(roc_auc_score(y_valid, y_pred)))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)
테스트 정확도: 0.5711
LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=500, n_jobs=-1, num_leaves=31, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, 

### 2. TF-IDF

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
def tokenizer(text):
    return text.split(',')

########### 사용할 모수들 ###################
min_df = 2
ngram_range = (1,1)
seed = 42

######################## 데이터로부터 단어가방 생성 ####################
vectorizer0 = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range)
revised_reviews0 = [' '.join(review.split(',')) for review in mytrain.text0]
vector_reviews0 = vectorizer0.fit_transform(revised_reviews0)
print(vector_reviews0.shape)

vectorizer1 = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range)
revised_reviews1 = [' '.join(review.split(',')) for review in mytrain.text1]
vector_reviews1 = vectorizer1.fit_transform(revised_reviews1)
print(vector_reviews1.shape)

player0 = pd.DataFrame(   vector_reviews0.toarray()*1, columns = vectorizer0.get_feature_names()  )
player0 = player0.add_prefix('player0_')
player1 = pd.DataFrame(   vector_reviews1.toarray()*1, columns = vectorizer1.get_feature_names()  )
player1 = player1.add_prefix('player1_')
X = pd.concat([player0,player1],axis=1)
y = mytrain.winner
print(X.shape)
#del vectorizer0, vectorizer1, revised_reviews0, revised_reviews1, vector_reviews0, vector_reviews1
#del player0, player1


(38872, 171)
(38872, 171)
(38872, 342)


In [0]:
#del X_train, X_valid, y_train, y_valid
########### 사용할 모수들 ###################
min_df = 2
ngram_range = (1,1)
seed = 42

######################## 데이터로부터 단어가방 생성 ####################
vectorizer0 = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range)
revised_reviews0 = [' '.join(review.split(',')) for review in mytest.text0]
vector_reviews0 = vectorizer0.fit_transform(revised_reviews0)
print(vector_reviews0.shape)

vectorizer1 = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range)
revised_reviews1 = [' '.join(review.split(',')) for review in mytest.text1]
vector_reviews1 = vectorizer1.fit_transform(revised_reviews1)
print(vector_reviews1.shape)

player0 = pd.DataFrame(   vector_reviews0.toarray()*1, columns = vectorizer0.get_feature_names()  )
player0 = player0.add_prefix('player0_')
player1 = pd.DataFrame(   vector_reviews1.toarray()*1, columns = vectorizer1.get_feature_names()  )
player1 = player1.add_prefix('player1_')

#del vectorizer0, vectorizer1, revised_reviews0, revised_reviews1, vector_reviews0, vector_reviews1

X_test = pd.concat([player0,player1],axis=1)
print(X_test.shape)

#del player0, player1

(16787, 165)
(16787, 166)
(16787, 331)


In [0]:
print(len(X.columns))
print(len(X_test.columns))
print(  len(    set(X.columns) & set(X_test.columns)   ) )

use_col = list(    set(X.columns) & set(X_test.columns)   )

X = X.loc[:,use_col]
X_test = X_test.loc[:,use_col]

342
331
329


In [0]:

###################### 모델링 시작 #######################
X_train, X_valid, y_train, y_valid = train_test_split(X, y,   stratify=y, random_state=seed)
from sklearn.naive_bayes import BernoulliNB
#model_bern = BernoulliNB().fit(X_train, y_train)
from sklearn.naive_bayes import MultinomialNB
#MNB = MultinomialNB(alpha=1).fit(X_train,y_train)
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000, random_state=seed).fit(X_train, y_train)
print('랜포 학습완료')
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(random_state=seed).fit(X_train,y_train)
print('LGBM 학습완료')

랜포 학습완료
LGBM 학습완료


In [0]:
## 랜포 #  min_df=1 : 0.6660 vs min_df=2 : 6625
y_pred = rf.predict_proba(X_valid)[:,1]
print(rf)
print("테스트 정확도: {:.4f}".format(roc_auc_score(y_valid, y_pred)))

## LGBM #  min_df=1 : 0.6625 vs min_df=2 : 6668
y_pred = lgbm.predict_proba(X_valid)[:,1]
print(lgbm)
print("테스트 정확도: {:.4f}".format(roc_auc_score(y_valid, y_pred)))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)
테스트 정확도: 0.6668
LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000,

## (예측을 위함)테스트 데이터 단어가방 생성

In [0]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16787 entries, 0 to 16786
Columns: 453 entries, player1_3_9 to player0_16_13
dtypes: int64(453)
memory usage: 58.0 MB


In [0]:
X.head(2)

,player0_15_8,player1_5_14,player1_15_15,player0_5_10,player1_8_12,player1_9_7,player1_3_8,player0_8_12,player1_14_15,player1_4_12,player1_13_12,player0_3_13,player1_6_7,player1_10_4,player0_14_7,player0_5_8,player0_13_16,player1_12_7,player1_3_13,player0_4_15,player0_12_5,player0_16_2,player0_6_14,player1_5_13,player1_5_9,player1_7_6,player1_7_13,player0_6_6,player0_14_5,player0_16_4,player1_17_2,player0_12_13,player0_16_6,player1_6_5,player1_5_2,player0_6_11,player0_11_6,player0_5_2,player1_16_2,player0_16_8,...,player1_6_10,player0_3_2,player1_14_17,player1_4_4,player1_14_11,player1_4_9,player0_13_7,player0_13_12,player1_4_11,player1_5_11,player1_12_4,player0_16_11,player0_3_5,player0_5_4,player1_8_10,player0_10_8,player0_3_10,player1_12_14,player1_11_16,player0_14_8,player1_12_6,player1_15_17,player1_6_12,player1_8_9,player0_15_12,player1_15_12,player1_3_6,player0_8_8,player1_10_5,player1_15_4,player1_5_7,player0_16_3,player0_7_12,player0_15_14,player0_13_11,player1_15_16,player1_9_8,player1_15_8,player0_9_7,player1_15_9
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,61,3,0,0,5,2,0,0,0,0,55,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,11,0,0,0,0,0,0,0,61,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,0,0,0,0,0,...,0,0,9,41,5,0,0,0,0,0,0,0,0,0,0,0,0,0,13,0,0,0,0,0,0,26,22,0,0,0,0,0,0,10,0,0,1,0,0,0


In [0]:
X.to_pickle('train_Camera_10단위.pkl')
X_test.to_pickle('test_Camera_10단위.pkl')

In [0]:
pred = rf.predict_proba(X_test)[:,1]

submission = pd.read_csv('sample_submission.csv', index_col=0)
submission['winner'] = submission['winner'] + pred
submission.to_csv('제출/텍스트전처리4_Camera만으로_단어임베딩만으로.csv', encoding='utf8')
submission.head()

,winner
game_id,
38872,0.475
38873,0.513
38874,0.553
38875,0.284
38876,0.505
